In [9]:
import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense

# loading data
from keras.datasets import cifar10

(x_train, label_train), (x_test, label_test) = cifar10.load_data()
train_ds, test_ds = cifar10.load_data()

In [11]:
# info on the dataset 
print(f"x_train.shape: {x_train.shape}") # (50000, 32, 32, 3) *3: color channels
print(f"y_train.shape: {label_train.shape}") #(50000, 1)

print(f"x_test.shape: {x_test.shape}") # (10000, 32, 32, 3)
print(f"y_test.shape: {label_test.shape}") # (10000, 1)


x_train.shape: (50000, 32, 32, 3)
y_train.shape: (50000, 1)
x_test.shape: (10000, 32, 32, 3)
y_test.shape: (10000, 1)


In [4]:
def preprocess(x_train, y_train, x_test, y_test): # or just preprocess(data)

    #  from 1-256 to 0-1 #### Q.why not into -1 to 1?
    x_train, x_test = x_train / 255.0, x_test / 255.0
    
    # flatten the label values
    y_train, y_test = y_train.flatten(), y_test.flatten()

    return x_train, y_train, x_test, y_test

In [ ]:
def preprocess(dataset):

    # convert data from uint8 to float32
    dataset = dataset.map(lambda x: tf.cast(x, tf.float32))

    # flatten the images into vectors
    dataset = dataset.map(lambda x: tf.reshape(x, (-1,)))

    # input normalization, just bringing image values from range [0, 255] to [-1, 1]
    dataset = dataset.map(lambda img, target: ((img / 128.) - 1., target))

In [5]:
x_train, y_train, x_test, y_test = preprocess(x_train, label_train, x_test, label_test)
print(f"x_train.shape: {x_train.shape}") # (50000, 32, 32, 3) *3: color channels
print(f"y_train.shape: {label_train.shape}") #(50000,)

print(f"x_test.shape: {x_test.shape}") # (10000, 32, 32, 3)
print(f"y_test.shape: {label_test.shape}") # (10000,)

x_train.shape: (50000, 32, 32, 3)
y_train.shape: (50000,)
x_test.shape: (10000, 32, 32, 3)
y_test.shape: (10000,)


In [8]:
# straight from the lecture
class BasicConv(tf.keras.Model):
    def __init__(self):
        super(BasicConv, self).__init__()

        # input 32x32x3 with 3 as the color channels
        self.convlayer1 = tf.keras.layers.Conv2D(filters=24, kernel_size=3, padding='same', activation='relu') # after this: 32x32x24
        self.convlayer2 = tf.keras.layers.Conv2D(filters=24, kernel_size=3, padding='same', activation='relu') # 32x32x24
        self.pooling = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2) # 16x16x24

        self.convlayer3 = tf.keras.layers.Conv2D(filters=48, kernel_size=3, padding='same', activation='relu') # 16x16x48
        self.convlayer4 = tf.keras.layers.Conv2D(filters=48, kernel_size=3, padding='same', activation='relu') # 16x16x48
        self.global_pool = tf.keras.layers.GlobalAvgPool2D() # 1x1x48

        self.out = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, x):
        x = self.convlayer1(x)
        x = self.convlayer2(x)
        x = self.pooling(x)
        x = self.convlayer3(x)
        x = self.convlayer4(x)
        x = self.global_pool(x)
        x = self.out(x)
        return x